In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
print(train.shape)
print(test.shape)
train.head(10)


In [ ]:
test.head(10)

In [ ]:
import ast

annot = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')
print(annot.shape)
annot.head()

In [ ]:
import tqdm
import os
import gc
import cv2
import glob
import numpy as np
import pandas as pd
from numba import cuda
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

RES = np.zeros( (512,512) )
mask = RES.copy()
mask[mask>0.5] = 1.
mask[mask<1] = 0
mask = mask.astype(np.uint8)
mask = np.stack( (mask,mask,mask), 2 )

del RES
gc.collect()
plt.imshow(mask)

# Chest X-Ray Visualisation

In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.vision.all import *

In [ ]:
df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")[['StudyInstanceUID', 'label']]
df['StudyInstanceUID'] = [i + ".jpg" for i in df['StudyInstanceUID']]
df.columns = ['name', 'label']
dls = ImageDataLoaders.from_df(df, "../input/ranzcr-clip-catheter-line-classification/", folder='train', valid_pct=0.2, item_tfms=Resize(224))
dls

In [ ]:
dls.show_batch()

# Training

In [ ]:
# Build the CNN model with the pretrained resnet34
# Error rate = 1 - accuracy
learn = cnn_learner(dls, models.resnet34, metrics = [accuracy])
# Train the model on 5 epochs of data at the default learning rate
learn.fit_one_cycle(5)

In [ ]:
# Rebuild interpreter and replot confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
submit = False

if submit:
    test_preds = []
    for i in range(len(enet_type)):
        if enet_type[i] == 'resnet34':
            print('resnet34 loaded')
            model = models.resnet34(enet_type[i], out_dim=len(target_cols))
            model = model.to(device)
        model.load_state_dict(torch.load(model_path[i], map_location='cuda:0'))
        if tta:
            test_preds += [tta_inference_func(test_loader)]
        else:
            test_preds += [inference_func(test_loader)]

    submission = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
    submission[target_cols] = np.mean(test_preds, axis=0)
    submission.to_csv('submission.csv', index=False)
else:
    pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv').to_csv('submission.csv', index=False)